In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')
tokenizer.save_pretrained('Llama-2-7b-hf')

In [ ]:
from collections import defaultdict
import pandas as pd
import json
import re

dict_OOV_freq = defaultdict(int)
dict_splits = defaultdict(int)

with open('../../../../TxtInputFiles/EBM_input.txt','r') as f:
    for line in f:
        # data = json.loads(line)
        target_text = line.strip()
        target_text = re.sub(r'[^\w\s]', '', target_text)
        sws = tokenizer.tokenize(target_text)
        print(sws)
        i = 0
        while(i < len(sws)-1):
            if sws[i].startswith('▁'):
                if sws[i+1].startswith('▁'):
                    i+=1
                    continue
                else:
                    sw = []
                    while(True):
                        sw.append(sws[i])
                        i+=1
                        if i == len(sws) or sws[i].startswith('▁'):
                            break
                    print(sw)
                    dict_OOV_freq[''.join(sw).replace('▁','')] += 1
                    dict_splits[''.join(sw).replace('▁','')] = len(sw)

list_token, list_freq, list_split = list(), list(), list()        
for token in dict_OOV_freq:
    list_token.append(token)
    list_freq.append(dict_OOV_freq[token])
    list_split.append(dict_splits[token])

df = pd.DataFrame({'token': list_token, 'freq': list_freq, 'split': list_split})
df.to_csv(f'EBM_OOV.csv', index=False)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('./CHQ_Vocabs/25K_0.5_/')
# tokenizer.save_pretrained('./EBM-20K-1/')

In [ ]:
COUNT_OOV = 0
vocab = tokenizer.get_vocab()
for v in vocab:
    if vocab[v] < 258: continue
    sws = tokenizer.tokenize(v.replace('▁',' '))
    if len(sws) > 1 and not sws[1]==v:
        # if vocab[v] < 32000: print(v,'PRE',vocab[v],tokenizer.tokenize(v.replace('▁',' ')))
        if vocab[v] >= 32000: 
            print(v,'ADDED',vocab[v],tokenizer.tokenize(v.replace('▁',' ')))
            COUNT_OOV += 1

In [ ]:
COUNT_OOV

In [ ]:
(3156/12212 + 984/3610 + 2579/9330 + 715/2358)/4

In [ ]:
import json

with open('./IFT+CHQ_Train_20K.jsonl','r') as f:
    for line in f:
        data = json.loads(line)
        print(data['prompt'].replace('##Title','##Resp'))
        break

In [ ]:
import pandas as pd

model_path = "pritamdeka/PubMedBERT-mnli-snli-scinli-scitail-mednli-stsb"

from sentence_transformers import SentenceTransformer
model = SentenceTransformer(model_path)

df_BioASQ = pd.read_csv('../../../../../../Medical/PushToNeumann/CSV-Datasets/EBM-Train.csv')
text_PAC = open('../../../../TxtInputFiles/PAC_input.txt','r').readlines()
text_BioASQ = df_BioASQ['input_text'].tolist()

In [ ]:
BioASQ_Encodings = model.encode(text_BioASQ, convert_to_tensor=True)
BioASQ_Encodings = BioASQ_Encodings.detach().cpu().numpy()

In [ ]:
import numpy as np
pac_encodings = np.load('../Llama-2-7B-BioASQ/PAC_Encodings.npy')

In [ ]:
import numpy as np
import tqdm
dist_rep = np.zeros((len(text_PAC),len(text_BioASQ)))

for idx1,entry in enumerate(pac_encodings):
    if (idx1+1)%1000 == 0:
        print(f'Completed {idx1+1} entries')
    for idx2,train_vec in enumerate(BioASQ_Encodings):
        dist_rep[idx1,idx2] = np.linalg.norm(entry-train_vec)

In [ ]:
dist_rep_mean = dist_rep.mean(axis=1)
dist_rep_mean.shape

sorted_indices = dist_rep_mean.argsort()[:50000]

PAC_BioASQ = [text_PAC[idx] for idx in sorted_indices]
with open('./EBM_PAC.txt','w') as f:
    f.write(''.join(PAC_BioASQ))
f.close()